Check for GPU


In [1]:
!nvidia-smi

Wed Dec  4 20:31:07 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.94                 Driver Version: 560.94         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060      WDDM  |   00000000:01:00.0  On |                  N/A |
|  0%   39C    P5             N/A /  115W |    3130MiB /   8188MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Set up directory

In [2]:
import os
HOME = os.getcwd()
print(HOME)

c:\Users\sondr\Documents\NTNU\9_semester\prosjekt\scripts


Load ultralytics


In [1]:
import ultralytics
import torchvision

ultralytics.checks()
torchvision.__version__

Ultralytics 8.3.15  Python-3.8.20 torch-2.3.0 CPU (AMD Ryzen 5 7500F 6-Core Processor)
Setup complete  (12 CPUs, 15.7 GB RAM, 335.0/930.8 GB disk)


'0.18.1a0'

HOME: Train it on custom dataset

In [ ]:
!yolo task=detect mode=train model=yolo11s.pt data=data.yaml epochs=50 imgsz=424 plots=True optimizer='Adam' dropout=0.2 patience=10

WARNING ⚠️ Python>=3.10 is required, but Python==3.8.20 is currently installed 
New https://pypi.org/project/ultralytics/8.3.44 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.15 🚀 Python-3.8.20 torch-2.3.0 CPU (AMD Ryzen 5 7500F 6-Core Processor)
engine\trainer: task=detect, mode=train, model=yolo11s.pt, data=data.yaml, epochs=50, time=None, patience=10, batch=16, imgsz=424, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train8, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.2, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, c


  0%|          | 0.00/18.4M [00:00<?, ?B/s]
  3%|▎         | 640k/18.4M [00:00<00:02, 6.30MB/s]
  9%|▉         | 1.62M/18.4M [00:00<00:02, 6.99MB/s]
 15%|█▍        | 2.75M/18.4M [00:00<00:01, 8.90MB/s]
 22%|██▏       | 4.00M/18.4M [00:00<00:01, 10.4MB/s]
 28%|██▊       | 5.12M/18.4M [00:00<00:01, 10.4MB/s]
 37%|███▋      | 6.75M/18.4M [00:00<00:00, 12.4MB/s]
 43%|████▎     | 8.00M/18.4M [00:00<00:00, 12.5MB/s]
 51%|█████     | 9.38M/18.4M [00:00<00:00, 12.8MB/s]
 58%|█████▊    | 10.8M/18.4M [00:00<00:00, 13.0MB/s]
 66%|██████▌   | 12.1M/18.4M [00:01<00:00, 13.1MB/s]
 73%|███████▎  | 13.5M/18.4M [00:01<00:00, 13.1MB/s]
 81%|████████  | 14.9M/18.4M [00:01<00:00, 13.1MB/s]
 88%|████████▊ | 16.2M/18.4M [00:01<00:00, 13.2MB/s]
 96%|█████████▌| 17.6M/18.4M [00:01<00:00, 13.1MB/s]
100%|██████████| 18.4M/18.4M [00:01<00:00, 12.1MB/s]

train: Scanning C:\Users\sondr\Documents\NTNU\9_semester\prosjekt\synthetic_data\training_424x240\labels\train.cache... 1966 images, 0 backgrounds, 0 corrupt: 1

: 

HOME: Apply to test set



In [ ]:
!yolo task=detect mode=predict model={HOME}/runs/detect/train_s/weights/best.pt conf=0.25 source={HOME}/../3_test_1min_hamkam_from_start/img1 save=True

Certainly! Integrating the pose estimation directly into your existing `RealSenseCamera` class and automatically setting the camera intrinsics in the `PointCloudBuilder` based on the RealSense SDK's retrieved values is a more efficient and streamlined approach. This eliminates the complexity of managing multiple streams and ensures that your system adapts seamlessly to various stream configurations.

Below is an in-depth, step-by-step guide to achieving this integration:

## **1. Architectural Overview**

### **a. Unified Class Design**

We'll consolidate functionalities into two main classes:

1. **`RealSenseCamera` Class**: Handles camera initialization, frame acquisition (color, depth, pose), YOLO-based object detection, point cloud integration, and display management.
2. **`PointCloudBuilder` Class**: Manages point cloud integration, leveraging camera pose data to build and maintain a global point cloud.

### **b. File Structure**

Organize your project directory as follows:

```
your_project/
│
├── main.py
├── realsense_camera.py
├── point_cloud_builder.py
├── pose_estimator.py  # Optional, if further abstraction is needed
├── yolo_model.py
├── utilities.py
└── config.py
```

- **`main.py`**: Entry point that ties all components together.
- **`realsense_camera.py`**: Contains the `RealSenseCamera` class.
- **`point_cloud_builder.py`**: Contains the `PointCloudBuilder` class.
- **`yolo_model.py` & `utilities.py`**: Existing modules for YOLO detection and utility functions.
- **`config.py`**: Configuration settings.

## **2. Dependencies and Setup**

### **a. Install Required Libraries**

Ensure you have the following libraries installed:

- **Open3D**: For point cloud processing.
- **pyrealsense2**: For interfacing with Intel RealSense cameras.
- **numpy**: For numerical operations.
- **opencv-python**: For image processing.
- **matplotlib**: For color mapping.
- **logging**: For logging events.

You can install these using `pip`:

```bash
pip install open3d pyrealsense2 numpy opencv-python matplotlib
```

**Note**: Ensure that the **Intel RealSense SDK** is installed and properly set up on your system. Follow the [Intel RealSense SDK installation guide](https://github.com/IntelRealSense/librealsense/blob/master/doc/distribution_linux.md) if needed.

### **b. Configuration File (`config.py`)**

Create a `config.py` file to manage all configuration settings. We'll remove the camera intrinsics from the configuration since they'll be dynamically retrieved from the RealSense camera.

```python
# config.py
import os

class Config:
    # Camera settings
    color_stream_width = 640
    color_stream_height = 480
    color_stream_fps = 30
    depth_stream_width = 640
    depth_stream_height = 480
    depth_stream_fps = 30

    # YOLO settings
    model_path = "path_to_yolo_model.pt"  # Update with your YOLO model path
    confidence_threshold = 0.5

    # Saving settings
    save_dir = "saved_frames"

    # Point Cloud settings
    voxel_size = 0.02  # Voxel size for downsampling
```

**Note**: Replace `"path_to_yolo_model.pt"` with the actual path to your YOLO model.

## **3. Implementing the Modules**

### **a. Point Cloud Builder (`point_cloud_builder.py`)**

The `PointCloudBuilder` class now accepts an **Open3D `PinholeCameraIntrinsic`** object directly, which will be provided by the `RealSenseCamera` class after retrieving the camera intrinsics.

```python
# point_cloud_builder.py
import open3d as o3d
import numpy as np
from typing import Tuple
import logging

class PointCloudBuilder:
    """
    Builds and maintains a global point cloud using incoming depth and color frames with pose data.
    """

    def __init__(self, intrinsic: o3d.camera.PinholeCameraIntrinsic, voxel_size: float = 0.02):
        """
        Initializes the PointCloudBuilder.

        Args:
            intrinsic (o3d.camera.PinholeCameraIntrinsic): Camera intrinsic parameters.
            voxel_size (float, optional): Voxel size for downsampling. Defaults to 0.02.
        """
        self.global_pcd = o3d.geometry.PointCloud()
        self.voxel_size = voxel_size
        self.intrinsic = intrinsic

    def integrate_frame(self, depth_frame: np.ndarray, color_frame: np.ndarray, pose: np.ndarray) -> None:
        """
        Integrates a single frame into the global point cloud.

        Args:
            depth_frame (np.ndarray): Depth image.
            color_frame (np.ndarray): Color image.
            pose (np.ndarray): 4x4 transformation matrix representing camera pose.
        """
        try:
            # Convert depth frame to Open3D format
            o3d_depth = o3d.geometry.Image(depth_frame)
            o3d_color = o3d.geometry.Image(color_frame)

            # Create RGBD image
            rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
                o3d_color,
                o3d_depth,
                convert_rgb_to_intensity=False
            )

            # Create point cloud from RGBD image
            pcd = o3d.geometry.PointCloud.create_from_rgbd_image(
                rgbd_image,
                self.intrinsic
            )

            # Transform point cloud to global frame
            pcd.transform(pose)

            # Downsample for efficiency
            pcd = pcd.voxel_down_sample(voxel_size=self.voxel_size)

            # Merge with global point cloud
            self.global_pcd += pcd
            self.global_pcd = self.global_pcd.voxel_down_sample(voxel_size=self.voxel_size)

            logging.info("Integrated new frame into global point cloud.")
        except Exception as e:
            logging.error(f"Error integrating frame into point cloud: {e}")

    def get_global_point_cloud(self) -> o3d.geometry.PointCloud:
        """
        Returns the current global point cloud.

        Returns:
            o3d.geometry.PointCloud: The global point cloud.
        """
        return self.global_pcd

    def visualize_global_point_cloud(self):
        """
        Visualizes the global point cloud using Open3D visualization.
        """
        try:
            o3d.visualization.draw_geometries([self.global_pcd])
        except Exception as e:
            logging.error(f"Error during point cloud visualization: {e}")

    def export_point_cloud(self, filename: str):
        """
        Exports the global point cloud to a PLY file.

        Args:
            filename (str): The filename for the exported point cloud.
        """
        try:
            self.global_pcd.downsample(voxel_size=self.voxel_size).to_file(filename)
            logging.info(f"Exported point cloud to {filename}.")
        except Exception as e:
            logging.error(f"Error exporting point cloud: {e}")
```

**Explanation:**

- **Initialization**: The constructor now accepts an `intrinsic` object of type `open3d.camera.PinholeCameraIntrinsic` and a `voxel_size` for downsampling.
  
- **Frame Integration**:
  - Converts depth and color frames to Open3D images.
  - Creates an RGB-D image and generates a point cloud from it.
  - Transforms the point cloud using the provided camera pose to align it in the global coordinate system.
  - Downsamples for efficiency and merges it into the global point cloud.

### **b. RealSense Camera (`realsense_camera.py`)**

The `RealSenseCamera` class is updated to:

1. Retrieve camera intrinsics from the RealSense SDK.
2. Create an `Open3D` intrinsic object.
3. Initialize the `PointCloudBuilder` with these intrinsics.
4. Integrate pose estimation directly within the frame acquisition loop.

```python
# realsense_camera.py
import os
import cv2
import numpy as np
import pyrealsense2 as rs
import datetime
import time
from enum import Enum, auto
from typing import List, Dict, Tuple
from yolo_model import YoloModel
from utilities import pad_to_square
import matplotlib.pyplot as plt
import logging
from config import Config
from point_cloud_builder import PointCloudBuilder
import open3d as o3d

# Define StreamView Enum
class StreamView(Enum):
    COLOR = auto()
    DEPTH = auto()
    DEPTH_COLORMAP = auto()
    COLOR_OVERLAY = auto()
    DEPTH_OVERLAY = auto()

class RealSenseCamera:
    """
    Encapsulates RealSense camera functionalities, including frame acquisition,
    YOLO object detection, point cloud integration, and display management.
    """

    def __init__(self, config: Config):
        """
        Initializes the RealSenseCamera with the given configuration.

        Args:
            config (Config): Configuration settings.
        """
        self.config = config
        self.pipeline = rs.pipeline()
        self.align = rs.align(rs.stream.color)
        self.profile = None
        self.depth_scale = None
        self.depth_intrinsics = None
        self.model = None
        self.is_running = False
        self.save_dir = self.config.save_dir
        os.makedirs(self.save_dir, exist_ok=True)

        # Initialize YOLO model (to be done after retrieving intrinsics)
        self.model = None

        # PointCloudBuilder will be initialized after retrieving intrinsics
        self.point_cloud_builder = None

    def _enable_stream(self, rs_config: rs.config) -> None:
        """
        Enables depth, color, and pose streams based on the configuration.

        Args:
            rs_config (rs.config): RealSense configuration object.
        """
        rs_config.enable_stream(
            rs.stream.depth,
            self.config.depth_stream_width,
            self.config.depth_stream_height,
            rs.format.z16,
            self.config.depth_stream_fps
        )
        rs_config.enable_stream(
            rs.stream.color,
            self.config.color_stream_width,
            self.config.color_stream_height,
            rs.format.bgr8,
            self.config.color_stream_fps
        )
        # Enable pose stream
        rs_config.enable_stream(rs.stream.pose)

    def _initialize_yolo_model(self) -> None:
        """
        Initializes the YOLO model based on the configuration.
        """
        self.model = YoloModel(self.config.model_path, self.config.confidence_threshold)
        self.model.load_model()
        logging.info("YOLO model initialized.")

    def initialize(self) -> None:
        """
        Initializes the camera with the settings from the configuration.
        """
        rs_config = rs.config()
        self._enable_stream(rs_config)
        
        # Start streaming
        self.profile = self.pipeline.start(rs_config)
        depth_sensor = self.profile.get_device().first_depth_sensor()
        self.depth_scale = depth_sensor.get_depth_scale()
        depth_stream_profile = self.profile.get_stream(rs.stream.depth).as_video_stream_profile()
        self.depth_intrinsics = depth_stream_profile.get_intrinsics()

        # Update camera intrinsics in config (optional, if you need to access it elsewhere)
        self.config.intrinsic_fx = self.depth_intrinsics.fx
        self.config.intrinsic_fy = self.depth_intrinsics.fy
        self.config.intrinsic_cx = self.depth_intrinsics.ppx
        self.config.intrinsic_cy = self.depth_intrinsics.ppy

        logging.info(f"Depth scale: {self.depth_scale} meters/unit.")
        logging.info(f"Depth intrinsics: {self.depth_intrinsics}.")

        # Create Open3D intrinsic object
        open3d_intrinsic = o3d.camera.PinholeCameraIntrinsic()
        open3d_intrinsic.set_intrinsics(
            self.depth_intrinsics.width,
            self.depth_intrinsics.height,
            self.depth_intrinsics.fx,
            self.depth_intrinsics.fy,
            self.depth_intrinsics.ppx,
            self.depth_intrinsics.ppy
        )

        # Initialize PointCloudBuilder with the retrieved intrinsics
        self.point_cloud_builder = PointCloudBuilder(intrinsic=open3d_intrinsic, voxel_size=self.config.voxel_size)
        logging.info("PointCloudBuilder initialized with camera intrinsics.")

        # Initialize YOLO model
        self._initialize_yolo_model()
        self.is_running = True

    def get_frames(self, pad: bool = False) -> Tuple[np.ndarray, np.ndarray, rs.pose_data]:
        """
        Retrieves depth, color, and pose frames from the camera.

        Args:
            pad (bool, optional): Whether to pad the images to square. Defaults to False.

        Returns:
            Tuple[np.ndarray, np.ndarray, rs.pose_data]: Depth image, color image, and pose data.
        """
        frames = self.pipeline.wait_for_frames()
        aligned_frames = self.align.process(frames)

        depth_frame = aligned_frames.get_depth_frame()
        color_frame = aligned_frames.get_color_frame()
        pose_frame = aligned_frames.get_pose_frame()

        if not depth_frame or not color_frame or not pose_frame:
            logging.warning("Incomplete frames received.")
            return None, None, None

        depth_image = np.asanyarray(depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())
        pose_data = pose_frame.get_pose_data()

        if pad:
            depth_image, _, _ = pad_to_square(depth_image)
            color_image, _, _ = pad_to_square(color_image)

        return depth_image, color_image, pose_data

    def _pose_to_matrix(self, pose: rs.pose_data) -> np.ndarray:
        """
        Converts RealSense pose data to a 4x4 transformation matrix.

        Args:
            pose (rs.pose_data): Pose data from RealSense.

        Returns:
            np.ndarray: 4x4 transformation matrix.
        """
        # Translation
        translation = np.array([pose.translation.x, pose.translation.y, pose.translation.z])

        # Rotation (quaternion)
        rot = pose.rotation
        rotation = self._quaternion_to_matrix(rot)

        # Create transformation matrix
        pose_matrix = np.identity(4)
        pose_matrix[:3, :3] = rotation
        pose_matrix[:3, 3] = translation

        return pose_matrix

    @staticmethod
    def _quaternion_to_matrix(q: rs.vector) -> np.ndarray:
        """
        Converts a quaternion to a rotation matrix.

        Args:
            q (rs.vector): Quaternion components.

        Returns:
            np.ndarray: 3x3 rotation matrix.
        """
        x, y, z, w = q.x, q.y, q.z, q.w
        xx, yy, zz = x * x, y * y, z * z
        xy, xz, yz = x * y, x * z, y * z
        wx, wy, wz = w * x, w * y, w * z

        rotation = np.array([
            [1 - 2*(yy + zz),     2*(xy - wz),       2*(xz + wy)],
            [2*(xy + wz),         1 - 2*(xx + zz),   2*(yz - wx)],
            [2*(xz - wy),         2*(yz + wx),       1 - 2*(xx + yy)]
        ])
        return rotation

    def apply_yolo(self, color_image: np.ndarray) -> Dict[StreamView, np.ndarray]:
        """
        Applies the YOLO model to the current color frame and draws bounding boxes.

        Args:
            color_image (np.ndarray): Color image.

        Returns:
            Dict[StreamView, np.ndarray]: Processed frames for different views.
        """
        if color_image is None:
            logging.warning("No color image available for YOLO.")
            return {}

        # Run inference on color input
        results = self.model.predict(color_image)

        # Draw bounding boxes
        color_overlay = self.model.draw_detections(
            image=color_image.copy(),
            detections=results
        )

        return {
            StreamView.COLOR: color_image,
            StreamView.COLOR_OVERLAY: color_overlay
            # You can add more overlays if needed
        }

    def prepare_display(self, processed_frames: Dict[StreamView, np.ndarray], views: List[StreamView], scale_factor: float = 1.5) -> np.ndarray:
        """
        Stacks and resizes requested frames for display.

        Args:
            processed_frames (Dict[StreamView, np.ndarray]): Processed frames.
            views (List[StreamView]): List of views to display.
            scale_factor (float, optional): Scaling factor for the display image. Defaults to 1.5.

        Returns:
            np.ndarray: The combined image ready for display.
        """
        if not views:
            raise ValueError("At least one view must be requested.")

        # Collect frames to display
        frames_to_display = [processed_frames[view] for view in views if view in processed_frames]
        if not frames_to_display:
            logging.warning("No frames available for display.")
            return np.array([])

        # Ensure all images have the same height
        min_height = min(frame.shape[0] for frame in frames_to_display)
        resized_frames = [cv2.resize(frame, (int(frame.shape[1] * min_height / frame.shape[0]), min_height)) for frame in frames_to_display]

        # Stack images horizontally
        combined_image = np.hstack(resized_frames)

        # Resize
        width = int(combined_image.shape[1] * scale_factor)
        height = int(combined_image.shape[0] * scale_factor)

        display_image = cv2.resize(combined_image, (width, height), interpolation=cv2.INTER_LINEAR)
        return display_image

    def save_frame(self, processed_frames: Dict[StreamView, np.ndarray], views: List[StreamView]) -> None:
        """
        Saves specified frames to disk.

        Args:
            processed_frames (Dict[StreamView, np.ndarray]): Processed frames.
            views (List[StreamView]): List of views to save.
        """
        try:
            timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S_%f")
            for view in views:
                frame = processed_frames.get(view)
                if frame is not None:
                    filename = f"{view.name.lower()}_{timestamp}.png"
                    filepath = os.path.join(self.save_dir, filename)
                    cv2.imwrite(filepath, frame)
                    logging.info(f"Saved {view.name} frame to {filepath}.")
                else:
                    logging.warning(f"{view.name} frame is not available.")
        except Exception as e:
            logging.error(f"Error saving frame: {e}")

    def run(self, views: List[StreamView] = [StreamView.COLOR, StreamView.COLOR_OVERLAY], window_name: str = "Realsense Camera", scale_factor: float = 1.5) -> None:
        """
        Runs the camera loop to display and process frames.

        Args:
            views (List[StreamView], optional): List of views to display. Defaults to [StreamView.COLOR, StreamView.DEPTH_COLORMAP].
            window_name (str, optional): Name of the display window. Defaults to "Realsense Camera".
            scale_factor (float, optional): Scaling factor for the display image. Defaults to 1.5.
        """

        last_save = 0
        save_cooldown = 0.5

        try:
            while self.is_running:
                # Get frames
                depth_image, color_image, pose_data = self.get_frames()

                if depth_image is None or color_image is None or pose_data is None:
                    continue

                # Convert pose data to transformation matrix
                pose_matrix = self._pose_to_matrix(pose_data)

                # Integrate into point cloud
                self.point_cloud_builder.integrate_frame(depth_image, color_image, pose_matrix)

                # Apply YOLO to the color image
                processed_frames = self.apply_yolo(color_image)

                # Prepare display
                display_image = self.prepare_display(processed_frames, views, scale_factor)
                if display_image.size == 0:
                    continue

                cv2.imshow(window_name, display_image)

                key = cv2.waitKey(1) & 0xFF
                current_time = time.time()
                if key == ord('q'):
                    logging.info("Exit key pressed.")
                    break
                elif key == ord('s'):
                    if (current_time - last_save) > save_cooldown:
                        self.save_frame(processed_frames, views)
                        last_save = current_time

                # Optional: Visualize global point cloud periodically
                # For example, every 100 frames
                # if frame_count % 100 == 0:
                #     self.point_cloud_builder.visualize_global_point_cloud()

        except KeyboardInterrupt:
            logging.info("Interrupted by user.")
        finally:
            self.stop()

    def stop(self) -> None:
        """
        Stops the camera and releases all resources.
        """
        self.is_running = False
        self.pipeline.stop()
        cv2.destroyAllWindows()
        logging.info("Camera pipeline stopped and windows closed.")
```

**Key Changes and Explanations:**

1. **Automatic Intrinsics Retrieval**:
   - Upon starting the pipeline, the camera intrinsics are retrieved from the depth stream using RealSense SDK's capabilities.
   - These intrinsics are then converted into an `Open3D` `PinholeCameraIntrinsic` object, which is required by the `PointCloudBuilder`.
   
2. **Initialization Order**:
   - The `PointCloudBuilder` is now initialized **after** retrieving the camera intrinsics, ensuring accurate point cloud generation regardless of the stream configuration.
   
3. **Dependency Management**:
   - Removed the dependence of `PointCloudBuilder` on `config.py` for intrinsics.
   - The `PointCloudBuilder` strictly relies on the passed `intrinsic` object, promoting better modularity and adaptability to different camera setups.
   
4. **Pose Estimation Integration**:
   - Pose data is directly retrieved within the `get_frames` method, eliminating the need for a separate `PoseEstimator` class.
   - The pose data is converted into a transformation matrix and used for point cloud integration.

### **c. Main Entry Point (`main.py`)**

Update your `main.py` to initialize and run the `RealSenseCamera`:

```python
# main.py
import logging
from realsense_camera import RealSenseCamera
from config import Config
import open3d as o3d

def main():
    # Configure logging
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

    # Initialize configuration
    config = Config()

    # Initialize RealSense camera
    camera = RealSenseCamera(config)
    camera.initialize()

    # Run camera
    camera.run()

    # After stopping, optionally visualize the global point cloud
    global_pcd = camera.point_cloud_builder.get_global_point_cloud()
    if global_pcd.has_points():
        logging.info("Visualizing the global point cloud.")
        o3d.visualization.draw_geometries([global_pcd])
    else:
        logging.info("No point cloud data to visualize.")

if __name__ == "__main__":
    main()
```

**Explanation:**

- **Logging**: Sets up logging to track events and potential issues.
- **Initialization**: Creates an instance of `RealSenseCamera` with the provided configuration and initializes it.
- **Execution**: Starts the camera processing loop.
- **Post-Processing**: After exiting the loop, visualizes the global point cloud if available using Open3D.

## **4. Additional Considerations**

### **a. Handling Camera Intrinsics**

Since the intrinsics are automatically retrieved from the RealSense camera upon initialization, you no longer need to manually set them in the configuration. This ensures that the `PointCloudBuilder` adapts to any stream setup.

**Note**: Ensure that your RealSense camera is correctly calibrated. Incorrect or imprecise intrinsics can lead to distorted or inaccurate point clouds.

### **b. Performance Optimization**

Real-time point cloud integration and YOLO-based object detection can be computationally intensive. To optimize performance:

1. **Downsample Depth Frames**: Reduce the resolution of depth frames if not needed at full resolution.
2. **Voxel Downsampling**: Adjust the `voxel_size` in `PointCloudBuilder` to balance detail and performance.
3. **Multi-Threading**: Consider running point cloud processing in a separate thread to prevent blocking the main camera loop.
4. **Hardware Acceleration**: Utilize GPU-based processing if available.

### **c. Error Handling and Robustness**

Ensure that all methods include robust error handling to gracefully manage unexpected issues, such as dropped frames or sensor disconnections.

### **d. Saving and Managing Point Clouds**

Depending on your application, you might want to:

- **Save Point Cloud Periodically**: Implement functionality to save snapshots of the global point cloud at regular intervals.
- **Manage Memory**: Implement strategies to limit the size of the global point cloud, such as removing distant points or maintaining a fixed memory footprint.

### **e. Enhancing Pose Estimation with Advanced SLAM**

While RealSense's built-in pose tracking is robust, integrating advanced SLAM solutions like **ORB-SLAM3** can further improve accuracy, especially in feature-poor environments. However, this requires more complex integration and potentially using system-level bindings or separate processes.

**Recommendation**: Start with RealSense's spatial tracking. Evaluate its performance in your application before considering more advanced SLAM integrations.

## **5. Complete Code Overview**

Below is the complete code for the updated modules based on the integration of pose estimation into the `RealSenseCamera` class and automatic setting of intrinsics in the `PointCloudBuilder`.

### **a. Configuration (`config.py`)**

```python
# config.py
import os

class Config:
    # Camera settings
    color_stream_width = 640
    color_stream_height = 480
    color_stream_fps = 30
    depth_stream_width = 640
    depth_stream_height = 480
    depth_stream_fps = 30

    # YOLO settings
    model_path = "path_to_yolo_model.pt"  # Update with your YOLO model path
    confidence_threshold = 0.5

    # Saving settings
    save_dir = "saved_frames"

    # Point Cloud settings
    voxel_size = 0.02  # Voxel size for downsampling
```

### **b. Point Cloud Builder (`point_cloud_builder.py`)**

```python
# point_cloud_builder.py
import open3d as o3d
import numpy as np
from typing import Tuple
import logging

class PointCloudBuilder:
    """
    Builds and maintains a global point cloud using incoming depth and color frames with pose data.
    """

    def __init__(self, intrinsic: o3d.camera.PinholeCameraIntrinsic, voxel_size: float = 0.02):
        """
        Initializes the PointCloudBuilder.

        Args:
            intrinsic (o3d.camera.PinholeCameraIntrinsic): Camera intrinsic parameters.
            voxel_size (float, optional): Voxel size for downsampling. Defaults to 0.02.
        """
        self.global_pcd = o3d.geometry.PointCloud()
        self.voxel_size = voxel_size
        self.intrinsic = intrinsic

    def integrate_frame(self, depth_frame: np.ndarray, color_frame: np.ndarray, pose: np.ndarray) -> None:
        """
        Integrates a single frame into the global point cloud.

        Args:
            depth_frame (np.ndarray): Depth image.
            color_frame (np.ndarray): Color image.
            pose (np.ndarray): 4x4 transformation matrix representing camera pose.
        """
        try:
            # Convert depth frame to Open3D format
            o3d_depth = o3d.geometry.Image(depth_frame)
            o3d_color = o3d.geometry.Image(color_frame)

            # Create RGBD image
            rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
                o3d_color,
                o3d_depth,
                convert_rgb_to_intensity=False
            )

            # Create point cloud from RGBD image
            pcd = o3d.geometry.PointCloud.create_from_rgbd_image(
                rgbd_image,
                self.intrinsic
            )

            # Transform point cloud to global frame
            pcd.transform(pose)

            # Downsample for efficiency
            pcd = pcd.voxel_down_sample(voxel_size=self.voxel_size)

            # Merge with global point cloud
            self.global_pcd += pcd
            self.global_pcd = self.global_pcd.voxel_down_sample(voxel_size=self.voxel_size)

            logging.info("Integrated new frame into global point cloud.")
        except Exception as e:
            logging.error(f"Error integrating frame into point cloud: {e}")

    def get_global_point_cloud(self) -> o3d.geometry.PointCloud:
        """
        Returns the current global point cloud.

        Returns:
            o3d.geometry.PointCloud: The global point cloud.
        """
        return self.global_pcd

    def visualize_global_point_cloud(self):
        """
        Visualizes the global point cloud using Open3D visualization.
        """
        try:
            o3d.visualization.draw_geometries([self.global_pcd])
        except Exception as e:
            logging.error(f"Error during point cloud visualization: {e}")

    def export_point_cloud(self, filename: str):
        """
        Exports the global point cloud to a PLY file.

        Args:
            filename (str): The filename for the exported point cloud.
        """
        try:
            self.global_pcd.downsample(voxel_size=self.voxel_size).to_file(filename)
            logging.info(f"Exported point cloud to {filename}.")
        except Exception as e:
            logging.error(f"Error exporting point cloud: {e}")
```

### **c. RealSense Camera (`realsense_camera.py`)**

```python
# realsense_camera.py
import os
import cv2
import numpy as np
import pyrealsense2 as rs
import datetime
import time
from enum import Enum, auto
from typing import List, Dict, Tuple
from yolo_model import YoloModel
from utilities import pad_to_square
import matplotlib.pyplot as plt
import logging
from config import Config
from point_cloud_builder import PointCloudBuilder
import open3d as o3d

# Define StreamView Enum
class StreamView(Enum):
    COLOR = auto()
    DEPTH = auto()
    DEPTH_COLORMAP = auto()
    COLOR_OVERLAY = auto()
    DEPTH_OVERLAY = auto()

class RealSenseCamera:
    """
    Encapsulates RealSense camera functionalities, including frame acquisition,
    YOLO object detection, point cloud integration, and display management.
    """

    def __init__(self, config: Config):
        """
        Initializes the RealSenseCamera with the given configuration.

        Args:
            config (Config): Configuration settings.
        """
        self.config = config
        self.pipeline = rs.pipeline()
        self.align = rs.align(rs.stream.color)
        self.profile = None
        self.depth_scale = None
        self.depth_intrinsics = None
        self.model = None
        self.is_running = False
        self.save_dir = self.config.save_dir
        os.makedirs(self.save_dir, exist_ok=True)

        # Initialize YOLO model (to be done after retrieving intrinsics)
        self.model = None

        # PointCloudBuilder will be initialized after retrieving intrinsics
        self.point_cloud_builder = None

    def _enable_stream(self, rs_config: rs.config) -> None:
        """
        Enables depth, color, and pose streams based on the configuration.

        Args:
            rs_config (rs.config): RealSense configuration object.
        """
        rs_config.enable_stream(
            rs.stream.depth,
            self.config.depth_stream_width,
            self.config.depth_stream_height,
            rs.format.z16,
            self.config.depth_stream_fps
        )
        rs_config.enable_stream(
            rs.stream.color,
            self.config.color_stream_width,
            self.config.color_stream_height,
            rs.format.bgr8,
            self.config.color_stream_fps
        )
        # Enable pose stream
        rs_config.enable_stream(rs.stream.pose)

    def _initialize_yolo_model(self) -> None:
        """
        Initializes the YOLO model based on the configuration.
        """
        self.model = YoloModel(self.config.model_path, self.config.confidence_threshold)
        self.model.load_model()
        logging.info("YOLO model initialized.")

    def initialize(self) -> None:
        """
        Initializes the camera with the settings from the configuration.
        """
        rs_config = rs.config()
        self._enable_stream(rs_config)
        
        # Start streaming
        self.profile = self.pipeline.start(rs_config)
        depth_sensor = self.profile.get_device().first_depth_sensor()
        self.depth_scale = depth_sensor.get_depth_scale()
        depth_stream_profile = self.profile.get_stream(rs.stream.depth).as_video_stream_profile()
        self.depth_intrinsics = depth_stream_profile.get_intrinsics()

        # Update camera intrinsics in config (optional, if you need to access it elsewhere)
        self.config.intrinsic_fx = self.depth_intrinsics.fx
        self.config.intrinsic_fy = self.depth_intrinsics.fy
        self.config.intrinsic_cx = self.depth_intrinsics.ppx
        self.config.intrinsic_cy = self.depth_intrinsics.ppy

        logging.info(f"Depth scale: {self.depth_scale} meters/unit.")
        logging.info(f"Depth intrinsics: {self.depth_intrinsics}.")

        # Create Open3D intrinsic object
        open3d_intrinsic = o3d.camera.PinholeCameraIntrinsic()
        open3d_intrinsic.set_intrinsics(
            self.depth_intrinsics.width,
            self.depth_intrinsics.height,
            self.depth_intrinsics.fx,
            self.depth_intrinsics.fy,
            self.depth_intrinsics.ppx,
            self.depth_intrinsics.ppy
        )

        # Initialize PointCloudBuilder with the retrieved intrinsics
        self.point_cloud_builder = PointCloudBuilder(intrinsic=open3d_intrinsic, voxel_size=self.config.voxel_size)
        logging.info("PointCloudBuilder initialized with camera intrinsics.")

        # Initialize YOLO model
        self._initialize_yolo_model()
        self.is_running = True

    def get_frames(self, pad: bool = False) -> Tuple[np.ndarray, np.ndarray, rs.pose_data]:
        """
        Retrieves depth, color, and pose frames from the camera.

        Args:
            pad (bool, optional): Whether to pad the images to square. Defaults to False.

        Returns:
            Tuple[np.ndarray, np.ndarray, rs.pose_data]: Depth image, color image, and pose data.
        """
        frames = self.pipeline.wait_for_frames()
        aligned_frames = self.align.process(frames)

        depth_frame = aligned_frames.get_depth_frame()
        color_frame = aligned_frames.get_color_frame()
        pose_frame = aligned_frames.get_pose_frame()

        if not depth_frame or not color_frame or not pose_frame:
            logging.warning("Incomplete frames received.")
            return None, None, None

        depth_image = np.asanyarray(depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())
        pose_data = pose_frame.get_pose_data()

        if pad:
            depth_image, _, _ = pad_to_square(depth_image)
            color_image, _, _ = pad_to_square(color_image)

        return depth_image, color_image, pose_data

    def _pose_to_matrix(self, pose: rs.pose_data) -> np.ndarray:
        """
        Converts RealSense pose data to a 4x4 transformation matrix.

        Args:
            pose (rs.pose_data): Pose data from RealSense.

        Returns:
            np.ndarray: 4x4 transformation matrix.
        """
        # Translation
        translation = np.array([pose.translation.x, pose.translation.y, pose.translation.z])

        # Rotation (quaternion)
        rot = pose.rotation
        rotation = self._quaternion_to_matrix(rot)

        # Create transformation matrix
        pose_matrix = np.identity(4)
        pose_matrix[:3, :3] = rotation
        pose_matrix[:3, 3] = translation

        return pose_matrix

    @staticmethod
    def _quaternion_to_matrix(q: rs.vector) -> np.ndarray:
        """
        Converts a quaternion to a rotation matrix.

        Args:
            q (rs.vector): Quaternion components.

        Returns:
            np.ndarray: 3x3 rotation matrix.
        """
        x, y, z, w = q.x, q.y, q.z, q.w
        xx, yy, zz = x * x, y * y, z * z
        xy, xz, yz = x * y, x * z, y * z
        wx, wy, wz = w * x, w * y, w * z

        rotation = np.array([
            [1 - 2*(yy + zz),     2*(xy - wz),       2*(xz + wy)],
            [2*(xy + wz),         1 - 2*(xx + zz),   2*(yz - wx)],
            [2*(xz - wy),         2*(yz + wx),       1 - 2*(xx + yy)]
        ])
        return rotation

    def apply_yolo(self, color_image: np.ndarray) -> Dict[StreamView, np.ndarray]:
        """
        Applies the YOLO model to the current color frame and draws bounding boxes.

        Args:
            color_image (np.ndarray): Color image.

        Returns:
            Dict[StreamView, np.ndarray]: Processed frames for different views.
        """
        if color_image is None:
            logging.warning("No color image available for YOLO.")
            return {}

        # Run inference on color input
        results = self.model.predict(color_image)

        # Draw bounding boxes
        color_overlay = self.model.draw_detections(
            image=color_image.copy(),
            detections=results
        )

        return {
            StreamView.COLOR: color_image,
            StreamView.COLOR_OVERLAY: color_overlay
            # You can add more overlays if needed
        }

    def prepare_display(self, processed_frames: Dict[StreamView, np.ndarray], views: List[StreamView], scale_factor: float = 1.5) -> np.ndarray:
        """
        Stacks and resizes requested frames for display.

        Args:
            processed_frames (Dict[StreamView, np.ndarray]): Processed frames.
            views (List[StreamView]): List of views to display.
            scale_factor (float, optional): Scaling factor for the display image. Defaults to 1.5.

        Returns:
            np.ndarray: The combined image ready for display.
        """
        if not views:
            raise ValueError("At least one view must be requested.")

        # Collect frames to display
        frames_to_display = [processed_frames[view] for view in views if view in processed_frames]
        if not frames_to_display:
            logging.warning("No frames available for display.")
            return np.array([])

        # Ensure all images have the same height
        min_height = min(frame.shape[0] for frame in frames_to_display)
        resized_frames = [cv2.resize(frame, (int(frame.shape[1] * min_height / frame.shape[0]), min_height)) for frame in frames_to_display]

        # Stack images horizontally
        combined_image = np.hstack(resized_frames)

        # Resize
        width = int(combined_image.shape[1] * scale_factor)
        height = int(combined_image.shape[0] * scale_factor)

        display_image = cv2.resize(combined_image, (width, height), interpolation=cv2.INTER_LINEAR)
        return display_image

    def save_frame(self, processed_frames: Dict[StreamView, np.ndarray], views: List[StreamView]) -> None:
        """
        Saves specified frames to disk.

        Args:
            processed_frames (Dict[StreamView, np.ndarray]): Processed frames.
            views (List[StreamView]): List of views to save.
        """
        try:
            timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S_%f")
            for view in views:
                frame = processed_frames.get(view)
                if frame is not None:
                    filename = f"{view.name.lower()}_{timestamp}.png"
                    filepath = os.path.join(self.save_dir, filename)
                    cv2.imwrite(filepath, frame)
                    logging.info(f"Saved {view.name} frame to {filepath}.")
                else:
                    logging.warning(f"{view.name} frame is not available.")
        except Exception as e:
            logging.error(f"Error saving frame: {e}")

    def run(self, views: List[StreamView] = [StreamView.COLOR, StreamView.COLOR_OVERLAY], window_name: str = "Realsense Camera", scale_factor: float = 1.5) -> None:
        """
        Runs the camera loop to display and process frames.

        Args:
            views (List[StreamView], optional): List of views to display. Defaults to [StreamView.COLOR, StreamView.DEPTH_COLORMAP].
            window_name (str, optional): Name of the display window. Defaults to "Realsense Camera".
            scale_factor (float, optional): Scaling factor for the display image. Defaults to 1.5.
        """

        last_save = 0
        save_cooldown = 0.5

        try:
            while self.is_running:
                # Get frames
                depth_image, color_image, pose_data = self.get_frames()

                if depth_image is None or color_image is None or pose_data is None:
                    continue

                # Convert pose data to transformation matrix
                pose_matrix = self._pose_to_matrix(pose_data)

                # Integrate into point cloud
                self.point_cloud_builder.integrate_frame(depth_image, color_image, pose_matrix)

                # Apply YOLO to the color image
                processed_frames = self.apply_yolo(color_image)

                # Prepare display
                display_image = self.prepare_display(processed_frames, views, scale_factor)
                if display_image.size == 0:
                    continue

                cv2.imshow(window_name, display_image)

                key = cv2.waitKey(1) & 0xFF
                current_time = time.time()
                if key == ord('q'):
                    logging.info("Exit key pressed.")
                    break
                elif key == ord('s'):
                    if (current_time - last_save) > save_cooldown:
                        self.save_frame(processed_frames, views)
                        last_save = current_time

                # Optional: Visualize global point cloud periodically
                # For example, every 100 frames
                # if frame_count % 100 == 0:
                #     self.point_cloud_builder.visualize_global_point_cloud()

        except KeyboardInterrupt:
            logging.info("Interrupted by user.")
        finally:
            self.stop()

    def stop(self) -> None:
        """
        Stops the camera and releases all resources.
        """
        self.is_running = False
        self.pipeline.stop()
        cv2.destroyAllWindows()
        logging.info("Camera pipeline stopped and windows closed.")
```

**Key Changes and Explanations:**

1. **Automatic Intrinsics Retrieval and Conversion**:
   - After starting the pipeline, the camera intrinsics are retrieved from the depth stream.
   - These intrinsics are then converted into an `Open3D` `PinholeCameraIntrinsic` object, which is passed to the `PointCloudBuilder` upon its initialization.
  
2. **PointCloudBuilder Initialization**:
   - The `PointCloudBuilder` is initialized **after** retrieving the camera intrinsics, ensuring accurate point cloud generation regardless of the stream configuration.
  
3. **Pose Estimation Integration**:
   - Pose data is directly retrieved within the `get_frames` method, eliminating the need for a separate `PoseEstimator` class.
   - The pose data is converted into a transformation matrix and used for point cloud integration.

4. **Method Adjustments**:
   - The `apply_yolo` method now accepts the `color_image` as an argument, ensuring that YOLO processing is tightly coupled with the frame acquisition.
   - The `prepare_display` method ensures that all frames have consistent heights before stacking them, improving the display output quality.

### **d. Main Entry Point (`main.py`)**

```python
# main.py
import logging
from realsense_camera import RealSenseCamera
from config import Config
import open3d as o3d

def main():
    # Configure logging
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

    # Initialize configuration
    config = Config()

    # Initialize RealSense camera
    camera = RealSenseCamera(config)
    camera.initialize()

    # Run camera
    camera.run()

    # After stopping, optionally visualize the global point cloud
    global_pcd = camera.point_cloud_builder.get_global_point_cloud()
    if global_pcd.has_points():
        logging.info("Visualizing the global point cloud.")
        o3d.visualization.draw_geometries([global_pcd])
    else:
        logging.info("No point cloud data to visualize.")

if __name__ == "__main__":
    main()
```

**Explanation:**

- **Logging**: Sets up logging to track events and potential issues.
- **Initialization**: Creates an instance of `RealSenseCamera` with the provided configuration and initializes it.
- **Execution**: Starts the camera processing loop.
- **Post-Processing**: After exiting the loop, visualizes the global point cloud if available using Open3D.

## **6. Execution and Testing**

1. **Ensure Correct Intrinsics**:
   - The `RealSenseCamera` class automatically retrieves intrinsics from the camera. Ensure that your RealSense camera is properly calibrated.
  
2. **Verify YOLO Model**:
   - Ensure that the YOLO model path in `config.py` is correct and that the `YoloModel` class correctly handles loading and inference.
  
3. **Run the Application**:
  
   ```bash
   python main.py
   ```
  
4. **Interact with the Application**:
   - **Display Frames**: A window will display the color image with YOLO detections.
   - **Save Frames**: Press the `s` key to save the current frames. Saved images will be stored in the `saved_frames` directory.
   - **Exit**: Press the `q` key to exit the application.

5. **Visualize Point Cloud**:
   - After exiting, the global point cloud will be visualized using Open3D's visualization window.

**Example Execution Flow:**

- Upon running `main.py`, the system initializes the RealSense camera, retrieves intrinsics, initializes the YOLO model and `PointCloudBuilder`.
- The application enters a loop where it continuously:
  - Captures depth, color, and pose frames.
  - Applies YOLO-based object detection on the color frames.
  - Integrates the depth and color frames into the global point cloud using the camera's pose.
  - Displays the processed color frames with detections.
  - Handles user inputs for saving frames or exiting.
- Upon exiting, the accumulated global point cloud is visualized.

## **7. Potential Enhancements**

### **a. Multi-Threaded Processing**

To further optimize performance, implement multi-threading to handle frame acquisition, point cloud integration, and YOLO-based object detection in parallel.

**Example Using Python's `threading` Module:**

```python
import threading

class RealSenseCamera:
    # ... existing methods ...

    def run(self, views: List[StreamView] = [StreamView.COLOR, StreamView.COLOR_OVERLAY], window_name: str = "Realsense Camera", scale_factor: float = 1.5) -> None:
        """
        Runs the camera loop to display and process frames in a separate thread.
        """

        last_save = 0
        save_cooldown = 0.5

        def camera_loop():
            while self.is_running:
                # Get frames
                depth_image, color_image, pose_data = self.get_frames()

                if depth_image is None or color_image is None or pose_data is None:
                    continue

                # Convert pose data to transformation matrix
                pose_matrix = self._pose_to_matrix(pose_data)

                # Integrate into point cloud
                self.point_cloud_builder.integrate_frame(depth_image, color_image, pose_matrix)

                # Apply YOLO to the color image
                processed_frames = self.apply_yolo(color_image)

                # Prepare display
                display_image = self.prepare_display(processed_frames, views, scale_factor)
                if display_image.size == 0:
                    continue

                cv2.imshow(window_name, display_image)

                key = cv2.waitKey(1) & 0xFF
                current_time = time.time()
                if key == ord('q'):
                    logging.info("Exit key pressed.")
                    self.is_running = False
                    break
                elif key == ord('s'):
                    if (current_time - last_save) > save_cooldown:
                        self.save_frame(processed_frames, views)
                        last_save = current_time

        # Start the camera loop in a separate thread
        camera_thread = threading.Thread(target=camera_loop)
        camera_thread.start()
        camera_thread.join()

    # ... rest of the class ...
```

**Note**: Ensure thread safety when accessing shared resources. Python's Global Interpreter Lock (GIL) may limit the effectiveness of multi-threading for CPU-bound tasks. For better performance, consider using multi-processing or asynchronous programming.

### **b. Advanced Pose Estimation with SLAM**

Integrate advanced SLAM solutions like **ORB-SLAM3** for improved pose estimation accuracy, especially in feature-poor environments.

- **Pros**:
  - Enhanced accuracy.
  - Loop closure capabilities to reduce drift.

- **Cons**:
  - Complex integration with Python.
  - Additional dependencies and setup required.

**Recommendation**: Start with RealSense's spatial tracking. Evaluate its performance in your application before considering more advanced SLAM integrations.

### **c. Enhanced Object Detection**

Leverage the accumulated point cloud for more advanced object detection, such as:

- **3D Bounding Boxes**: Detect objects in 3D space using point cloud data.
- **Spatial Reasoning**: Improve object placement and spatial context understanding.

### **d. Data Export and Storage**

Implement functionality to export the global point cloud to common formats (e.g., PLY, LAS) for offline analysis or integration with other systems.

```python
# Example in main.py after running the camera loop
if global_pcd.has_points():
    camera.point_cloud_builder.export_point_cloud("global_point_cloud.ply")
```

### **e. User Interface Enhancements**

Develop a more sophisticated user interface to handle interactions, visualize point clouds in real-time, and manage application states using libraries like **PyQt** or **Tkinter**.

## **8. Conclusion**

By integrating pose estimation directly into the `RealSenseCamera` class and automatically retrieving and setting camera intrinsics for the `PointCloudBuilder`, you achieve a more streamlined, efficient, and adaptable system. This design ensures that your application can handle various stream configurations without manual adjustments, enhancing scalability and maintainability.

**Next Steps**:

1. **Implement the Provided Code**: Ensure that all modules are correctly implemented and integrated.
2. **Test the Application**: Run the application in a controlled environment to verify point cloud generation and YOLO-based detections.
3. **Optimize Performance**: Depending on the application's requirements, consider multi-threading or other optimization techniques.
4. **Iterate and Enhance**: Based on testing outcomes, iteratively improve the system by adding features or refining existing functionalities.

Feel free to reach out with any further questions or for assistance with specific implementation details!
